In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  

In [2]:
##pip install numpy

# 1. Reading the image

In [3]:
im = plt.imread(r'C:\Users\strocha\Documents\Python Scripts\Keilen\WTA_exp_2025-01-15T13-08-43.213.tif') ## Change figure file 
print(im.dtype)
print(im.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\strocha\\Documents\\Python Scripts\\Keilen\\WTA_exp_2025-01-15T13-08-43.213.tif'

In [ ]:
plt.imshow(im)

In [ ]:
# Load the grayscale image
from skimage.color import rgb2gray

# Convert to grayscale
im_gray = rgb2gray(im)

plt.imshow(im_gray)
print(im_gray.dtype)
print(im_gray.shape)

# 3. Adjust threshold

In [ ]:
from skimage.filters import try_all_threshold

fig, ax = try_all_threshold(im_gray, figsize=(10, 8), verbose=False)
plt.show()

In [ ]:
from skimage import io, filters, morphology
from scipy import ndimage
#Apply a threshold to create a binary mask
threshold = filters.threshold_minimum(im_gray)  ## change to filter that works the best: threshold_[filter](im_gray)
binary_mask = im_gray > threshold

plt.imshow(binary_mask)


In [ ]:
inverted_mask = 1 - binary_mask

# 4. Measurement and filtering

In [ ]:
import matplotlib.pyplot as plt
from skimage import measure, morphology
from pandas import DataFrame
import numpy as np

# Assuming binary_mask is the binary mask from the thresholded/smoothed image
labels = measure.label(inverted_mask)
props = measure.regionprops(labels)

# Create a list to store the properties of each labeled region
lr = []
for prop in props:
    # Collect properties for display and filtering
    aa = [prop.label, prop.area, prop.centroid, prop.major_axis_length, prop.minor_axis_length]
    lr.append(aa) 

# Create DataFrame and filter out small objects based on area
Outprops = DataFrame(lr, columns=['Original_Label', 'Area', 'Centroid', 'Length', 'Width'])
df3 = Outprops[Outprops['Area'] > 60 ]  # Filter out objects with area <= 100 ## Change area size

# Add a new column with sequential labels starting from 1
df3 = df3.reset_index(drop=True)  # Reset index to ensure numbering starts from 0
df3['Adjusted_Label'] = df3.index + 1  # Create a new column for adjusted labels

# Create a new binary mask that includes only the filtered regions
filtered_binary_mask = np.zeros_like(inverted_mask)
for _, row in df3.iterrows():
    original_label_id = row['Original_Label']
    filtered_binary_mask[labels == original_label_id] = 1

# Plot the filtered binary mask with adjusted labels
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(filtered_binary_mask, cmap='gray')
ax.set_title("Filtered Objects with Sequential Labels on Binary Mask")

# Loop through filtered DataFrame to plot adjusted labels on the filtered mask
for _, row in df3.iterrows():
    centroid = row['Centroid']
    adjusted_label = row['Adjusted_Label']
    # Plot the adjusted label number at the centroid location
    ax.text(centroid[1], centroid[0], str(adjusted_label), color='red', fontsize=12, ha='center', va='center')

plt.axis('off')
plt.savefig('filtered_objects_with_labels.png', bbox_inches='tight', dpi=300)


# 5. Display and save the dataFrame

In [ ]:
df3

In [ ]:
# save as csv, allow user to choose name  ## change file name
df3.to_csv('58A_exp_measure.csv')

# 6. Graphing data from single dataset

In [ ]:
# Compute Aspect Ratio
if 'Length' in df3.columns and 'Width' and 'Area' in df3.columns:
    df3['Aspect_Ratio'] = df3['Length'] / df3['Width']
    df3['Length (μm)'] = df3['Length'] / 6 # Depends on lens used. Our x40 lens is 6 pixels per micron
    df3['Width (μm)'] = df3['Width'] / 6 # Depends on lens used. Our x40 lens is 6 pixels per micron
    df3['Area (μm)'] = df3['Area'] / 6 **2 # Depends on lens used. Our x40 lens is 6 pixels per micron

# Define the features to plot
features = ['Length (μm)', 'Width (μm)', 'Aspect_Ratio', 'Area (μm)']

# Compute overall mean and standard deviation values
mean_values = df3[features].mean()
std_values = df3[features].std()

# Set style
sns.set_style("whitegrid")

# Create separate plots
for feature in features:
    plt.figure(figsize=(8, 5))
    sns.barplot(x=[feature], y=[mean_values[feature]], yerr=[std_values[feature]], capsize=0.2)
    plt.title(f'Average {feature} for Entire Dataset with Standard Deviation')
    plt.ylabel(f'{feature.replace("Mean ", "")}')
    plt.savefig(f"{feature.replace(' ', '_').lower()}_comparison.png", dpi=300, bbox_inches='tight')
    plt.show()
    plt.show()
